# **GENERATING NETCDF OUTPUTS** #

#### Objective: ####
+ Create [NetCDF](https://unidata.github.io/netcdf4-python/#netCDF4) files <span style="color:RoyalBlue">*almost?*</span> fully-complaint with [CF](https://cfconventions.org/) [conventions](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.10/cf-conventions.html)

---

In [188]:
import netCDF4 as nc4
import numpy as np

In [33]:
# help function

#~ CUSTOM.ROUNDING TO SPECIFIC.BASE ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# # https://stackoverflow.com/a/18666678/5885810
def ROUNDX(x, prec=3, base=PRECISION):
    return (base * (np.array(x) / base).round()).round(prec)

In [2]:
nc_file = 'for_wrong-1.nc'

data = nc4.Dataset( nc_file )

print( data)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): x(408), y(470)
    variables(dimensions): float64 x(x), float64 y(y), int32 xomethin(), float32 rain(y, x), uint8 mask(y, x)
    groups: 


In [5]:
data['rain']
data['rain'][:]


masked_array(
  data=[[  8.85314465,  12.18742561,  12.00609589, ...,  19.86124039,
          21.57285881,  22.31961441],
        [ 11.41961765,  12.11657238,  11.99657345, ...,  21.30685425,
          22.31961441,  22.31961441],
        [ 11.08961964,  12.00933456,  11.87809658, ...,  21.30685425,
          22.31961441,  22.31961441],
        ...,
        [461.05093384, 461.05093384, 462.56283569, ..., 530.9977417 ,
         516.53491211, 516.53491211],
        [472.34197998, 472.34197998, 470.32394409, ..., 530.9977417 ,
         516.53491211, 516.53491211],
        [459.64810181, 459.64810181, 462.9229126 , ..., 527.46783447,
         526.47857666, 526.47857666]],
  mask=False,
  fill_value=1e+20)

In [10]:
print( data['rain'][:].min() )
print( data['rain'][:].max() )

8.853144645690918
1527.81982421875


In [193]:

# OGC-WKT for HAD [taken from https://epsg.io/42106]
WKT_OGC = 'PROJCS["WGS84_/_Lambert_Azim_Mozambique",'\
    'GEOGCS["unknown",'\
        'DATUM["unknown",'\
            'SPHEROID["Normal Sphere (r=6370997)",6370997,0]],'\
        'PRIMEM["Greenwich",0,'\
            'AUTHORITY["EPSG","8901"]],'\
        'UNIT["degree",0.0174532925199433,'\
            'AUTHORITY["EPSG","9122"]]],'\
    'PROJECTION["Lambert_Azimuthal_Equal_Area"],'\
    'PARAMETER["latitude_of_center",5],'\
    'PARAMETER["longitude_of_center",20],'\
    'PARAMETER["false_easting",0],'\
    'PARAMETER["false_northing",0],'\
    'UNIT["metre",1,'\
        'AUTHORITY["EPSG","9001"]],'\
    'AXIS["Easting",EAST],'\
    'AXIS["Northing",NORTH],'\
    'AUTHORITY["EPSG","42106"]]'

DATE_ORIGIN    = '1970-01-01'                       # to store dates as INT
TIME_ZONE      = 'Africa/Addis_Ababa'               # Local Time Zone (see links below for more names)

# RAINFMT = 'f4'
RAINFMT = 'u2'                          # 'u' for UNSIGNED.INT  ||  'i' for SIGNED.INT  ||  'f' for FLOAT
                                        # number of Bytes (1, 2, 4 or 8) to store the RAINFALL variable (into)
TIMEINT = 'u4'                          # format for integers in TIME dimension
TIMEFIL = +(2**( int(TIMEINT[-1]) *8 )) -1

PRECISION = 0.025
iMIN = 8.

In [108]:
INTEGER = int( RAINFMT[-1] )

# # if one wants signed integers (e.g.)...
#     MINIMUM = -(2**(INTEGER *8 -1))       # -->     -32768  (smallest  signed integer of 16 Bits)
#     MAXIMUM = +(2**(INTEGER *8 -1)-1)     # -->     +32767  (largest   signed integer of 16 Bits)

# # if one wants un-signed integers (e.g.)...
MINIMUM = 1                             # -->          1  (1 because you need 0 for filling)
MAXIMUM = +(2**( INTEGER *8 )) -1       # -->      65535  (largest unsigned integer of 16 Bits -> 0 also counts)
# MAXIMUM = +(2**( 32 )) -1             # --> 4294967296  (largest unsigned integer of 32 Bits)

print( MINIMUM )
print( MAXIMUM )

1
65535


find precision... having previously established limits

In [119]:
# run your own (customized) tests
temp = 3.14159
seed = 4000 # -> a bit more than the maximum of RAIN data
# epsn = 0.05
epsilon = [0.05, 0.03, 0.025, 0.02, 0.01]

for epsn in epsilon:
    while temp > epsn:
        temp = (seed -iMIN) / (MAXIMUM -MINIMUM -1)
        seed = seed - 1
    # print(seed)
    
    print( (f'starting from {iMIN}, you\'d need a max. of ~{seed+1} to guarantee an epsilon of {epsn}') )


starting from 8.0, you'd need a max. of ~3284 to guarantee an epsilon of 0.05
starting from 8.0, you'd need a max. of ~1973 to guarantee an epsilon of 0.03
starting from 8.0, you'd need a max. of ~1646 to guarantee an epsilon of 0.025
starting from 8.0, you'd need a max. of ~1318 to guarantee an epsilon of 0.02
starting from 8.0, you'd need a max. of ~663 to guarantee an epsilon of 0.01


In [120]:
#~ TO SCALE 'DOWN' FLOATS TO INTEGERS ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

# NORMALIZING THE RAINFALL SO IT CAN BE STORED AS 16-BIT INTEGER (65,536 -> unsigned)
# https://stackoverflow.com/a/59193141/5885810      (scaling 'integers')
# https://stats.stackexchange.com/a/70808/354951    (normalize data 0-1)

# if you want a larger precision (or your variable is in the 'low' scale,
# ...say Summer Temperatures in Celsius) you must/could lower this limit.
iMAX = PRECISION *(MAXIMUM -MINIMUM -1) +iMIN
print( f'iMAX = {iMAX}' )
# 131.070 -> for MINIMUM==0
# 131.068 -> for MINIMUM==1

# SCALING FACTOR:
SCL = (iMAX - iMIN) / ( MAXIMUM -MINIMUM -1)   # -1 because 0 doesn't count

# ADDITIVE FACTOR:
# ADD = iMAX - SCL *MAXIMUM
ADD = iMIN - SCL *MINIMUM

print( SCL )
print( ADD )

iMAX = 1646.325
0.025
7.975


In [102]:

# testing
allv = PRECISION *(np.linspace(MINIMUM, MAXIMUM-1, MAXIMUM-1) -1) +iMIN
# allv = np.arange(iMIN, iMAX +PRECISION, PRECISION)
print( allv.shape )
# Out[22]: (65534,)

print( allv[-1] )
print( allv[-1] == iMAX )
# Out[22]: True

allt = ((allv - ADD) /SCL)
allt = allt.round(0).astype('u2')
print( np.where( np.diff(allt)!=1 ) )
# Out[22]: (array([], dtype=int64),)

vall = (allt *SCL) +ADD
vall = ROUNDX( (allt *SCL) +ADD )
print( vall )


(65534,)
1646.325
True
(array([], dtype=int64),)
[   8.       8.025    8.05  ... 1646.275 1646.3   1646.325]


In [145]:


print( len( data.dimensions['x'] ) )
print( len( data['x'] ) )



408
408


In [134]:

data['x'][1:10]
data['x'][1:10].data

array([1347500., 1352500., 1357500., 1362500., 1367500., 1372500.,
       1377500., 1382500., 1387500.])

In [148]:

data['mask'][:].data

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [137]:

import xarray as xr


In [163]:

import rioxarray as rio

from zoneinfo import ZoneInfo
from datetime import datetime, timezone#, timedelta
from dateutil.tz import tzlocal


In [141]:

# create VOID data

void = np.empty( (len( data.dimensions['y'] ), len( data.dimensions['x'] )) )
void.fill(np.nan)

# create xarray
xoid = xr.DataArray(data=void, dims=['y', 'x']#, name='void'
    , coords=dict(y=(['y'], data['y'][:].data), x=(['x'], data['x'][:].data), )
    , attrs=dict(_FillValue=np.nan, units='mm', ),
    )
xoid.rio.write_crs(rio.crs.CRS( WKT_OGC ), grid_mapping_name='spatial_ref', inplace=True)

print( xoid)
print( xoid.rio.crs )
print( ' '.join( map(str, np.roll(np.asarray(xoid.rio.transform()).reshape(3,3), shift=1, axis=1)[:-1].ravel().tolist() )) )


<xarray.DataArray (y: 470, x: 408)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * y            (y) float64 1.168e+06 1.162e+06 ... -1.172e+06 -1.178e+06
  * x            (x) float64 1.342e+06 1.348e+06 ... 3.372e+06 3.378e+06
    spatial_ref  int32 0
Attributes:
    _FillValue:  nan
    units:       mm
PROJCS["WGS84_/_Lambert_Azim_Mozambique",GEOGCS["unknown",DATUM["unknown",SPHEROID["Normal Sphere (r=6370997)",6370997,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",5],PARAMETER["longitude_of_center",20],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","42106"]]
1340000.0

In [176]:


date_now = datetime.now(tzlocal())
print( date_now )

date_now = date_now.replace( tzinfo=ZoneInfo( TIME_ZONE ) )       
print( date_now )


2023-10-03 11:14:17.762780+01:00
2023-10-03 11:14:17.762780+03:00


In [177]:


DATE_ORIGEN = datetime.strptime(DATE_ORIGIN, '%Y-%m-%d').replace(tzinfo=ZoneInfo( TIME_ZONE ))
print( DATE_ORIGEN )

date_sec = (date_now - DATE_ORIGEN).total_seconds()
print( date_sec )


1970-01-01 00:00:00+03:00
1696331657.76278


In [179]:

T_RES   =  30                           # in minutes! -> TEMPORAL.RES of TIME.SLICES
TIME_DICT_ = dict(seconds=1 ,minutes=1/60, hours=1/60**2, days=1/(60**2*24))
TIME_OUTNC = 'minutes'                  # UNITS (since DATE_ORIGIN) for NC.TIME dim


In [183]:

date_min = date_sec *TIME_DICT_[ TIME_OUTNC ]
print( date_min )


28272194.29604633


In [180]:

#~ ROUND TIME.STAMPS to the 'T_RES' FLOOR! (or NEAREST 'T_RES') ~~~~~~~~~~~~~~~#
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
def BASE_ROUND( stamps, base= T_RES ):
# https://stackoverflow.com/a/2272174/5885810
# "*60" because STAMPS come in seconds; and T_RES is in minutes
    base = base *TIME_DICT_[ TIME_OUTNC ] *60
    iround = (base *(np.ceil(stamps /base) -1))#.astype( TIMEINT )
# # activate line below if you want to the NEAREST 'T_RES' (instead of FLOOR)
#     iround = (base *(stamps /base).round())#.astype( TIMEINT )
    return iround

In [185]:


o_date_min = BASE_ROUND( date_min )
print( o_date_min )

28272180.0


In [197]:

out_name = 'fiv_ok.nc'
    
# CREATE NC.FILE
nc = nc4.Dataset(out_name, 'w', format='NETCDF4')
nc.created_on = datetime.now(tzlocal()).strftime('%Y-%m-%d %H:%M:%S %Z')#%z


# define SUB.GROUP and its dimensions
sub_grp = nc.createGroup( 'group_one' )

sub_grp.createDimension('y', len( data.dimensions['y'] ))
sub_grp.createDimension('x', len( data.dimensions['x'] ))
# sub_grp.createDimension('t', None)                  # unlimited

sref_name = 'spatial_ref'

grid = sub_grp.createVariable(sref_name, 'u1')
grid.long_name = sref_name
grid.crs_wkt = xoid.spatial_ref.attrs['crs_wkt']
grid.spatial_ref = xoid.spatial_ref.attrs['crs_wkt']
# https://www.simplilearn.com/tutorials/python-tutorial/list-to-string-in-python
# https://www.geeksforgeeks.org/how-to-delete-last-n-rows-from-numpy-array/
# grid.GeoTransform = ' '.join( map(str, list(xoid.rio.transform()) ) )
# # this is apparently the "correct" way to store the GEOTRANSFORM!
grid.GeoTransform = ' '.join( map(str, np.roll(np.asarray(xoid.rio.transform()).reshape(3,3),
    shift=1, axis=1)[:-1].ravel().tolist() ))                   # [:-1] removes last row

# ~~~ from CFCONVENTIONS.ORG ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ [start]
grid.grid_mapping_name = 'lambert_azimuthal_equal_area'
grid.latitude_of_projection_origin = 5
grid.longitude_of_projection_origin = 20
grid.false_easting = 0
grid.false_northing = 0
# grid.horizontal_datum_name = 'WGS84'                          # in pple this can also be un-commented!
grid.reference_ellipsoid_name = 'sphere'
grid.projected_crs_name = 'WGS84_/_Lambert_Azim_Mozambique'     # new in CF-1.7 [https://cfconventions.org/wkt-proj-4.html]
# ~~~ from CFCONVENTIONS.ORG ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ [end]

# ~~~ from https://publicwiki.deltares.nl/display/NETCDF/Coordinates [start]
grid._CoordinateTransformType = 'Projection'
grid._CoordinateAxisTypes = 'GeoY GeoX'
# ~~~ from https://publicwiki.deltares.nl/display/NETCDF/Coordinates ~ [end]

# # STORING LOCAL COORDINATES
yy = sub_grp.createVariable('projection_y_coordinate', 'i4', dimensions=('y'))#,chunksizes=CHUNK_3D( [ len(data['y']) ], valSize=4))
xx = sub_grp.createVariable('projection_x_coordinate', 'i4', dimensions=('x'))#,chunksizes=CHUNK_3D( [ len(data['x']) ], valSize=4))
yy[:] = data['y'][:].data
xx[:] = data['x'][:].data
yy.coordinates = 'projection_y_coordinate'
xx.coordinates = 'projection_x_coordinate'
yy.units = 'meter'
xx.units = 'meter'
yy.long_name = 'y coordinate of projection'
xx.long_name = 'x coordinate of projection'
yy._CoordinateAxisType = 'GeoY'
xx._CoordinateAxisType = 'GeoX'
yy.grid_mapping = sref_name
xx.grid_mapping = sref_name

tag_y = yy.getncattr('coordinates')
tag_x = xx.getncattr('coordinates')

# store the MASK
# mask_chunk = CHUNK_3D( [ len( data['y'] ), len( data['x'] ) ], valSize=1)
ncmask = sub_grp.createVariable('mask', 'i1', dimensions=('y','x'), zlib=True, complevel=9)#,chunksizes=mask_chunk#,fill_value=0
ncmask[:] = data['mask'][:].data.astype('i1')
ncmask.grid_mapping = sref_name
ncmask.long_name = 'catchment mask'
ncmask.description = '1 means catchment or region : 0 is void'
ncmask.coordinates = f'{tag_y} {tag_x}'

# # if ANOTHER/OTHER variable is needed (e.g.)
# maskxx = sub_grp.createVariable('k_means', datatype='i1', dimensions=('y','x'), dimensions=('y','x'), zlib=True, complevel=9,
#                                 fill_value=np.array( -1 ).astype('i1'))#,chunksizes=mask_chunk#,least_significant_digit=3)
# maskxx.grid_mapping = sref_name
# maskxx.long_name = 'k-means clusters'
# maskxx.description = '-1 indicates region out of any cluster'
# maskxx.coordinates = f'{tag_y} {tag_x}'


# define the TIME.variable/dimension
nctnam = 'time'
sub_grp.createDimension(nctnam, None)

timexx = sub_grp.createVariable(nctnam, TIMEINT, (nctnam), fill_value=TIMEFIL)#, chunksizes=chunkt)
timexx[:] = o_date_min.astype( TIMEINT )
timexx.long_name = 'starting time'
timexx.units = f"{TIME_OUTNC} since {DATE_ORIGEN.strftime('%Y-%m-%d %H:%M:%S')}"#'%Y-%m-%d %H:%M:%S %Z%z'
timexx.calendar = 'proleptic_gregorian'#'gregorian'#
timexx._CoordinateAxisType = 'Time'
timexx.coordinates = nctnam


if RAINFMT[0]=='f':
#-DOING.FLOATS------------------------------------------------------------------
    ncvarx = sub_grp.createVariable('rain', datatype=f'{RAINFMT}'
        ,dimensions=(nctnam,'y','x'), zlib=True, complevel=9, least_significant_digit=3, fill_value=np.nan)
    ncvarx[:] = (data['rain'][:].data).astype( f'{RAINFMT}' )
else:
#-DOING.INTEGERS--------------------------------------------------------------
    ncvarx = sub_grp.createVariable('rain', datatype=f'{RAINFMT}',
        dimensions=(nctnam,'y','x'), zlib=True, complevel=9,
        fill_value=np.array( 0 ).astype( f'{RAINFMT}' ))#, chunksizes=chunkx,least_significant_digit=3)
    ncvarx[0,:] = (( (data['rain'][:].data -ADD) /SCL ).round(0)).astype( f'{RAINFMT}' )
    # ncvarx[0,:] -> ONLY FOR THIS EXERCISE

ncvarx.precision = PRECISION
ncvarx.scale_factor = SCL
ncvarx.add_offset = ADD
# ncvarx._FillValue = np.array( MINIMUM ).astype( f'{RAINFMT}' )
# ncvarx._FillValue = np.array( 0 ).astype( f'{RAINFMT}' )
ncvarx.units = 'mm'
ncvarx.long_name = 'rainfall'
ncvarx.grid_mapping = sref_name
ncvarx.coordinates = f'{tag_y} {tag_x}'
#ncvarx.coordinates = f'{yy.getncattr("coordinates")} {xx.getncattr("coordinates")}'

nc.close()
